# Linear model of stock price changes during COP

In [1]:
from datetime import timedelta, datetime
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
from pathlib import Path
import seaborn as sns
from sklearn.model_selection import train_test_split

In [2]:
from scipy.stats import ttest_1samp, gstd, kstest, ttest_ind
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm

In [3]:
version = "15"
# This is a switch to allow investigation of multiple sets of companies. Valid options include:
# "renewable_all", "fossil_34", "control_34" (top 20 of any kind), "constsust_34" 
# (top 34 with neutral ethics ratings), "greencred_34", "fossil_100", 
# "all_companies" (top 100 with any ethics ratings, uses rating as a continuous variable)
# and a few assorted variables for one-off investigations.
filetype = "fossil_20"
# Only include companies that have data before this date:
require_time_start = pd.to_datetime('2011-01-01')
# Do we want to include a period some days before/after the dictionary of events?
# You will need padding for 1-day events (e.g. OPEC meetings, IPCC reports, OilSpills)
padafter = 0
padbefore = 0
# If this variable is not empty/False, we switch to studing a different time series. Options include None = COPs, 
# "OilSpill", "OPEC", "OPEC_Conference"
# "OPEC_28" (only 28 equally spaced OPEC meetings so stats are easier), "IPCC"
copOrOther = "OPEC_conf_28"
# If this is a string (probably "constsust_34"), we subtract the average fractional change in this filetype 
# before calculating statistics.
norm_group = "constsust_34"
# Do we want to remove data from days with stock splits and low volume? (remove days with vol<1000 and the day before and after)
skip_dodgy_days = True

In [4]:
if copOrOther:
    output = f"./output/version{version}/{copOrOther}/{filetype}/before{padbefore}_after{padafter}_norm{norm_group}"
else:
    output = f"./output/version{version}/{filetype}/before{padbefore}_after{padafter}_norm{norm_group}"
if skip_dodgy_days:
    output += "_cleaned"
Path(output).mkdir(exist_ok=True, parents=True)

In [5]:
# name of the relevant column in the company df. May be overwritten below depending on filetype. 
close = "Close"
# List of companies whose files we will read
if filetype == "renewable":
    companylist = [
        "0916.HK", "BEP", "EDPR.LS", "FSLR", "NEE", "NHPC.NS", "SUZLON.NS", "VWS.CO", "NPI.TO", "009830.KS"
    ]
elif filetype == "renewable_20":
    companylist = [
        "0916.HK", "BEP", "EDPR.LS", "FSLR", "NEE", "NHPC.NS", "SUZLON.NS", "VWS.CO", "NPI.TO", "009830.KS",
        "ORA", "3800.HK", "PLUG", "NDX1.F", "BLX.TO", "ECV.F", "SLR.MC", "S92.F", "VBK.F", "CSIQ", 
    ]
elif filetype == "fossil":
    companylist = [
        "XOM", "CVX", "SHEL", "601857.SS", "TTE", "COP", "BP", "PBR", "EQNR",  "600028.SS"
    ]
elif filetype == "fossil_20":
    companylist = [
        "XOM", "CVX", "SHEL", "601857.SS", "TTE", "COP", "BP", "PBR", "EQNR",  "600028.SS",
        "0883.HK", "SO", "ENB", "SLB", "DUK", "EOG", "CNQ", "EPD", "E", "OXY"
        
    ]
elif filetype == "constsust_20":
    companylist = ["ABT", "AMZN", "AZN", "BAC", "BRK-B", "COST", "GOOG", "JNJ", "JPM", "KO", 
                  "LLY", "MCD", "MRK", "NESN.SW", "NVO", "PEP", "PG", "ROG.SW", "TYT.L", "WMT"]
elif filetype == "control":
    companylist = ["AAPL", "AMZN", "BRK-B", "GOOG", "LLY", "MSFT", "NVDA", "TSM", "UNH", "V"]
elif filetype == "control_20":
    companylist = ["AAPL", "AMZN", "BRK-B", "GOOG", "LLY", "MSFT", "NVDA", "TSM", "UNH", "V",
                   "HD", "PG", "005930.KS", "MC.PA", "JNJ", "MA", "WMT", "AVGO", "NVO", "JPM"]
elif filetype == "oilprice":
    companylist = ["OilPrice"]
    close = "Adj Close**"
elif filetype == "tmp":
    companylist = ["^SPX"]
elif filetype == "oilfuturesApril":
    companylist = ["CrudeOilWTIFrontMonthApril"]
elif filetype == "greencred_20":
    companylist = ["005930.KS", "AAPL", "ACN", "ADBE", "AMD", "ASML", "AVGO", "CRM", "HD", "MA", 
                   "MC.PA", "MSFT", "NFLX", "NVDA", "NVS", "ORCL", "RMS.PA", "TMO", "UNH", "V"]
elif filetype == "dirty_20": 
    companylist = [
        'XOM', '600519.SS', 'CVX', 'KO', 'RELIANCE.NS',
        'SHEL', '601857.SS', 'WFC', '601288.SS', '601988.SS', 'BA', 'COP',
        'RIO', 'PBR', 'BP', '601088.SS', 'EQNR', 'MO', 'CNQ', 'ITC.NS'
    ]
else: 
    companylist = os.listdir(f"./input/{filetype}/")
    companylist = [x[:-4] for x in companylist]
    # In these cases, we expect there to be around 100 items
    if len(companylist) < 21:
        raise ValueError("Invalid filetype option")

In [6]:
if filetype[-2:] == "20":
    assert len(companylist) == 20
elif filetype == "all_companies":
    assert len(companylist) == 100

In [7]:
if norm_group:
    companynormlist = os.listdir(f"./input/{norm_group}/")
    companynormlist = [x[:-4] for x in companynormlist]
    try: 
        norm_string_end = int(norm_group.split("_")[-1])
    except:
        norm_string_end = None
    if norm_string_end:
        assert len(companynormlist) == norm_string_end

In [8]:
if not copOrOther:
    copdates = pd.read_csv("./input/CopDates.txt", delimiter="|")
    copdates = copdates.iloc[:, 1:-1]
    copdates.columns = copdates.columns.str.replace('\s+', '')
    copdates["Start"] = pd.to_datetime(copdates["Start"])
    copdates["End"] = pd.to_datetime(copdates["End"])
    meetingstring = "COP number"
    copOrOtherLongstring  = "COP"
elif copOrOther == "OPEC":
    copdates = pd.read_csv("./input/OPEC_all_2002.csv", delimiter=",", parse_dates=["Date"])
    copdates["Start"] = copdates["Date"]
    copdates["End"] = copdates["Date"]
    meetingstring = f"OPEC meeting"
    copOrOtherLongstring = meetingstring
elif copOrOther == "OPEC_Conference":
    copdates = pd.read_csv("./input/OPEC_all_2002.csv", delimiter=",", parse_dates=["Date"])
    copdates["Start"] = copdates["Date"]
    copdates["End"] = copdates["Date"]
    copdates = copdates.drop_duplicates(subset=["Date"], keep="first")
    copdates = copdates.loc[["Meeting of the OPEC Conference" in i for i in copdates["Meeting Title"]]]
    copdates = copdates.reset_index(drop=True)
    meetingstring = f"OPEC meeting"
    copOrOtherLongstring = meetingstring
elif copOrOther == "OPEC_28":
    copdates = pd.read_csv("./input/OPEC_all_2002.csv", delimiter=",", parse_dates=["Date"])
    copdates["Start"] = copdates["Date"]
    copdates["End"] = copdates["Date"]
    copdates = copdates.drop_duplicates(subset=["Date"], keep="first")
    copdates.iloc[np.arange(len(copdates) % 28, len(copdates),  (len(copdates) // 28 )), :].reset_index()
    meetingstring = f"OPEC meeting"
    copOrOtherLongstring = meetingstring
elif copOrOther == "OPEC_conf_28":
    copdates = pd.read_csv("./input/OPEC_all_2002.csv", delimiter=",", parse_dates=["Date"])
    copdates["Start"] = copdates["Date"]
    copdates["End"] = copdates["Date"]
    copdates = copdates.loc[["Meeting of the OPEC Conference" in i for i in copdates["Meeting Title"]]]
    copdates = copdates.drop_duplicates(subset=["Date"], keep="first")
    copdates.iloc[np.arange(len(copdates) % 28, len(copdates), (len(copdates) // 28 )), :].reset_index()
    meetingstring = f"OPEC meeting"
    copOrOtherLongstring = meetingstring
elif copOrOther == "OilSpill":
    copdates = pd.read_csv("./input/Oil spills data_v2.csv", delimiter=",", parse_dates=["Date"])
    copdates["Start"] = copdates["Date"]
    copdates["End"] = copdates["Date"]
    meetingstring = f"Oil spill"
    copOrOtherLongstring = meetingstring
elif copOrOther == "IPCC":
    copdates = pd.read_csv("./input/IPCC_dates.csv", delimiter=",", parse_dates=["Date"])
    copdates["Start"] = copdates["Date"]
    copdates["End"] = copdates["Date"]
    meetingstring = "IPCC report"
    copOrOtherLongstring = "IPCC report release"
else:
    raise ValueError("Did not specify a valid copOrOther")
if copOrOther:
    # In all cases, we can't allow duplicate dates so should  flush them out.
    copdates = copdates.drop_duplicates(subset=["Date"], keep="first")

In [43]:
pd.read_csv("./input/OPEC_all_2002.csv", delimiter=",", parse_dates=["Date"]).tail(40)

,Date,Meeting Title
67,2019-12-06,7th OPEC and non-OPEC Ministerial Meeting
68,2019-12-06,177th Meeting of the OPEC Conference
69,2019-12-06,7th OPEC and non-OPEC Ministerial Meeting
70,2020-03-05,178th (Extraordinary) Meeting of the OPEC Conf...
71,2020-04-09,9th (Extraordinary) OPEC and non-OPEC Minister...
72,2020-04-12,10th (Extraordinary) OPEC and non-OPEC Ministe...
73,2020-06-06,179th Meeting of the OPEC Conference
74,2020-06-06,11th OPEC and non-OPEC Ministerial Meeting
75,2020-11-30,180th Meeting of the OPEC Conference
76,2020-12-03,12th OPEC and non-OPEC Ministerial Meeting


In [9]:
if padbefore:
    copdates["Start"] = copdates["Start"] - timedelta(days=padbefore)
if padafter:
    copdates["End"] = copdates["End"] + timedelta(days=padafter)

# Loop over companies recording behaviour during COP or the same time gap shifted by some weeks

In [11]:
# This reads the data from a filestring ending "filetype/company.csv" and appends it to the list results
def read_company_data(filetype, company, results):
    file_path = f'./input/{filetype}/{company}.csv'# Name of the variable denoting price at close
    # Read the data from the CSV file into a DataFrame
    df = pd.read_csv(file_path, parse_dates=['Date'], dayfirst=True)
    df = df[np.isfinite(df[close])]
    df["company"] = company
    df["DayChange"] = df[close].pct_change()
    df["DayVar"] = (df["High"]-df["Low"]) / df["High"]
    df["COP"] = np.nan
    for num, row in copdates.iterrows():
            if (
                row["Start"] > df["Date"].min()
            ) & (row["End"] < df["Date"].max()
            ) & (sum(
                (df["Date"] >= row["Start"]) & (df["Date"] <= row["End"])
            ) > 1):
                df.loc[(row["Start"] <= df["Date"]) & (row["End"] >= df["Date"]), "COP"] = num
    results.append(df)

In [12]:
all_data = []
for company in companylist:
    read_company_data(filetype, company, all_data)
all_data = pd.concat(all_data)
all_data

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,company,DayChange,DayVar,COP
0,1990-01-02,3.919844,3.968965,3.870723,3.929668,5326000,0.0,0.0,XOM,NaN,0.024752,NaN
1,1990-01-03,3.919843,3.929667,3.860898,3.890370,4980400,0.0,0.0,XOM,-0.010000,0.017500,NaN
2,1990-01-04,3.880546,3.900195,3.811777,3.851074,6013200,0.0,0.0,XOM,-0.010101,0.022670,NaN
3,1990-01-05,3.851077,3.870725,3.801956,3.831428,3854800,0.0,0.0,XOM,-0.005101,0.017766,NaN
4,1990-01-08,3.831425,3.910019,3.821601,3.890370,4302000,0.0,0.0,XOM,0.015384,0.022613,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
8671,2024-06-04,60.009998,60.009998,59.000000,59.889999,6910100,0.0,0.0,OXY,-0.011227,0.016831,NaN
8672,2024-06-05,60.150002,60.150002,59.450001,59.840000,4903000,0.0,0.0,OXY,-0.000835,0.011638,NaN
8673,2024-06-06,59.750000,60.110001,59.520000,60.080002,5026800,0.0,0.0,OXY,0.004011,0.009815,NaN
8674,2024-06-07,59.810001,60.270000,59.330002,59.480000,6962700,0.0,0.0,OXY,-0.009987,0.015596,NaN


In [13]:
assert len(all_data[all_data.Date > pd.datetime(year=2011, month=1, day=1)]["company"].unique()) == len(companylist)

C:\Users\rlamboll\AppData\Local\Temp\ipykernel_3996\2877074325.py:1: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  assert len(all_data[all_data.Date > pd.datetime(year=2011, month=1, day=1)]["company"].unique()) == len(companylist)


# Optionally clean the data

In [14]:
if any(all_data[all_data.Close < 0]):
    print("Warning: data goes negative")
    print(all_data[all_data.Close < 0])
    all_data = all_data[all_data.Close > 0]

Empty DataFrame
Columns: [Date, Open, High, Low, Close, Volume, Dividends, Stock Splits, company, DayChange, DayVar, COP]
Index: []


In [15]:
all_data[((all_data["DayChange"] <-0.5) | (all_data["DayChange"] >0.5))]

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,company,DayChange,DayVar,COP
7604,2020-03-09,15.161045,18.673971,11.716238,12.173599,104930300,0.79,0.0,OXY,-0.520138,0.37259,NaN


In [16]:
all_data = all_data[((all_data["DayChange"] >-0.5) & (all_data["DayChange"] < 0.5))]

In [17]:
all_data[((all_data["DayChange"] <-0.2) | (all_data["DayChange"] >0.2))&((all_data["Volume"]<1000)|(all_data["Volume"].shift(1)<1000)|(all_data["Volume"].shift(-1)<1000))]

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,company,DayChange,DayVar,COP
1330,2006-10-10,1.900899,1.95,1.848291,1.879856,358216706,0.0,0.0,600028.SS,-0.210604,0.052158,NaN


In [18]:
if norm_group:
    normdata = []
    for company in companynormlist:
        read_company_data(norm_group, company, normdata)
    normdata = pd.concat(normdata)
    meanNorm = normdata.groupby('Date').mean()

In [19]:
if skip_dodgy_days:
    all_data_2 = all_data[
        (all_data["Volume"]>1000)&(all_data["Volume"].shift(1)>1000)&
        (all_data["Volume"].shift(-1)>1000)&(all_data["Stock Splits"]==0)
    ]
    deleted_data = all_data[
        ~((all_data["Volume"]>1000)&(all_data["Volume"].shift(1)>1000)&
        (all_data["Volume"].shift(-1)>1000)&(all_data["Stock Splits"]==0))
    ]
    all_data = all_data_2
    if norm_group:
        normdata[
            (normdata["Volume"]>1000)&(normdata["Volume"].shift(1)>1000)&
            (normdata["Volume"].shift(-1)>1000)&(normdata["Stock Splits"]==0)
        ]
        meanNorm = normdata.groupby('Date').mean()

In [20]:
# If we have data about ethics ratings we can also add this
if filetype == "all_companies":
    company_ethics = pd.read_csv("./input/companiesmarketcap.com - Companies ranked by Market Cap - CompaniesMarketCap.com.csv")
    company_ethics = company_ethics.loc[:, ["Symbol", "Sustainalytics value"]]
    all_data = pd.merge(all_data, company_ethics, left_on="company", right_on="Symbol")
    del all_data["Symbol"]
    company_ethics["Sustainalytics value"].hist()

In [21]:
all_cat = pd.get_dummies(all_data, drop_first=True)
all_cat["COP"] = [1 if x == x else 0 for x in all_cat["COP"]]
all_cat

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,DayChange,DayVar,...,company_EOG,company_EPD,company_EQNR,company_OXY,company_PBR,company_SHEL,company_SLB,company_SO,company_TTE,company_XOM
2,1990-01-04,3.880546,3.900195,3.811777,3.851074,6013200,0.0,0.0,-0.010101,0.022670,...,0,0,0,0,0,0,0,0,0,1
3,1990-01-05,3.851077,3.870725,3.801956,3.831428,3854800,0.0,0.0,-0.005101,0.017766,...,0,0,0,0,0,0,0,0,0,1
4,1990-01-08,3.831425,3.910019,3.821601,3.890370,4302000,0.0,0.0,0.015384,0.022613,...,0,0,0,0,0,0,0,0,0,1
5,1990-01-09,3.890372,3.900196,3.811778,3.811778,3485600,0.0,0.0,-0.020202,0.022670,...,0,0,0,0,0,0,0,0,0,1
6,1990-01-10,3.811780,3.841252,3.792132,3.831428,4522400,0.0,0.0,0.005155,0.012788,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8670,2024-06-03,62.220001,62.240002,60.189999,60.570000,7562900,0.0,0.0,-0.030880,0.032937,...,0,0,0,1,0,0,0,0,0,0
8671,2024-06-04,60.009998,60.009998,59.000000,59.889999,6910100,0.0,0.0,-0.011227,0.016831,...,0,0,0,1,0,0,0,0,0,0
8672,2024-06-05,60.150002,60.150002,59.450001,59.840000,4903000,0.0,0.0,-0.000835,0.011638,...,0,0,0,1,0,0,0,0,0,0
8673,2024-06-06,59.750000,60.110001,59.520000,60.080002,5026800,0.0,0.0,0.004011,0.009815,...,0,0,0,1,0,0,0,0,0,0


In [22]:
all_cat = all_cat[all_cat.Date > pd.datetime(year=1995, month=1, day=1)]
if norm_group:
    all_cat = pd.merge(all_cat, meanNorm.loc[:, ["DayChange", "DayVar"]].reset_index(), on="Date")
all_cat = all_cat[np.isnan(all_cat).sum(axis=1)==0]

C:\Users\rlamboll\AppData\Local\Temp\ipykernel_3996\4055006351.py:1: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  all_cat = all_cat[all_cat.Date > pd.datetime(year=1995, month=1, day=1)]


In [23]:
all_cat["Year"] = all_cat.Date.dt.year.astype(str)

In [24]:
all_cat = pd.get_dummies(all_cat)

In [25]:
# If we have data about ethics ratings we can also add an interaction between this and COPs
if filetype == "all_companies":
    all_cat["COP_good_ethics"] = all_cat["COP"] * (all_cat["Sustainalytics value"] < 20)
    all_cat["COP_mid_ethics"] = all_cat["COP"] * (all_cat["Sustainalytics value"] > 20) * (all_cat["Sustainalytics value"] < 30)
    all_cat["COP_bad_ethics"] = all_cat["COP"] * (all_cat["Sustainalytics value"] > 30)
    # Since all companies are covered by one of these, we remove the general case to prevent colinearity issues
    del all_cat["COP"]

In [26]:
if not norm_group:
    target = "DayChange"
    ignore_col = "DayVar"
    X_train = all_cat.loc[:, all_cat.columns != ignore_col].iloc[:, 9:]
else:
    target = "DayChange_x"
    ignore_col = "DayVar_y"
    X_train = all_cat.loc[:, all_cat.columns != ignore_col].iloc[:, 10:]
# We have established that this works so no longer use the test-train distinction
y_train = all_cat[target]

In [27]:
ls=sm.OLS(y_train,X_train).fit()

In [28]:
ls.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            DayChange_x   R-squared:                       0.223
Model:                            OLS   Adj. R-squared:                  0.223
Method:                 Least Squares   F-statistic:                     780.9
Date:                Wed, 10 Jul 2024   Prob (F-statistic):               0.00
Time:                        15:29:52   Log-Likelihood:             3.4675e+05
No. Observations:              133359   AIC:                        -6.934e+05
Df Residuals:                  133309   BIC:                        -6.929e+05
Df Model:                          49                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
COP               -3.685e-18    1.9e-20   -194.219      0.000   -3.72e-18   -3.65e-18
company_600028.SS    -0.0003      0.000     -0.837      0.403      -0.001       0.000
company_601857.SS    -0.0008      0.000     -2.000      0.046      -0.002   -1.53e-05
company_BP           -0.0004      0.000     -1.301      0.193      -0.001       0.000
company_CNQ           0.0002      0.000      0.443      0.658      -0.001       0.001
company_COP          -0.0002      0.000     -0.702      0.483      -0.001       0.000
company_CVX          -0.0003      0.000     -0.922      0.357      -0.001       0.000
company_DUK          -0.0004      0.000     -1.233      0.217      -0.001       0.000
company_E            -0.0003      0.000     -1.018      0.309      -0.001       0.000
company_ENB          -0.0001      0.000     -0.429      0.668      -0.001       0.001
company_EOG       -6.137e-05      0.000     -0.184      0.854      -0.001       0.001
company_EPD          -0.0002      0.000     -0.442      0.658      -0.001       0.001
company_EQNR         -0.0001      0.000     -0.394      0.694      -0.001       0.001
company_OXY          -0.0001      0.000     -0.382      0.703      -0.001       0.001
company_PBR        7.362e-05      0.000      0.212      0.832      -0.001       0.001
company_SHEL         -0.0004      0.000     -1.164      0.244      -0.001       0.000
company_SLB          -0.0003      0.000     -0.941      0.346      -0.001       0.000
company_SO           -0.0003      0.000     -1.034      0.301      -0.001       0.000
company_TTE          -0.0003      0.000     -0.931      0.352      -0.001       0.000
company_XOM          -0.0003      0.000     -1.006      0.315      -0.001       0.000
DayChange_y           0.9083      0.005    195.057      0.000       0.899       0.917
Year_1995             0.0001      0.000      0.253      0.800      -0.001       0.001
Year_1996             0.0004      0.000      0.995      0.320      -0.000       0.001
Year_1997            -0.0001      0.000     -0.354      0.723      -0.001       0.001
Year_1998            -0.0013      0.000     -3.133      0.002      -0.002      -0.000
Year_1999            -0.0001      0.000     -0.278      0.781      -0.001       0.001
Year_2000             0.0013      0.000      3.212      0.001       0.001       0.002
Year_2001             0.0003      0.000      0.822      0.411      -0.000       0.001
Year_2002             0.0005      0.000      1.232      0.218      -0.000       0.001
Year_2003             0.0004      0.000      1.031      0.303      -0.000       0.001
Year_2004             0.0009      0.000      2.413      0.016       0.000       0.002
Year_2005             0.0009      0.000      2.554      0.011       0.000       0.002
Year_2006             0.0006      0.000      1.655      0.098      -0.000

In [29]:
model = LinearRegression()
model.fit(X_train,y_train)

LinearRegression()

In [30]:
summary_str = ls.summary().as_text()
with open(output + "/OLSsummary.txt", "w") as f:
    f.write(summary_str)

In [31]:
# Now do the same for daily variability
if not norm_group:
    target = "DayVar"
    ignore_col = "DayChange"
    accept_index_dayvar = (all_cat[target] >= 0) & (all_cat[target] < 0.5)
    X2_train = all_cat.loc[accept_index_dayvar, all_cat.columns != ignore_col].iloc[:, 9:]
else:
    target = "DayVar_x"
    ignore_col = "DayChange_y"
    accept_index_dayvar = (all_cat[target] >= 0) & (all_cat[target] < 0.5)
    X2_train = all_cat.loc[accept_index_dayvar, all_cat.columns != ignore_col].iloc[:, 10:]
y2_train = all_cat.loc[accept_index_dayvar, target]

In [32]:
ls2=sm.OLS(y2_train, X2_train).fit()

In [33]:
ls2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               DayVar_x   R-squared:                       0.470
Model:                            OLS   Adj. R-squared:                  0.470
Method:                 Least Squares   F-statistic:                     2411.
Date:                Wed, 10 Jul 2024   Prob (F-statistic):               0.00
Time:                        15:29:55   Log-Likelihood:             4.1198e+05
No. Observations:              133356   AIC:                        -8.239e+05
Df Residuals:                  133306   BIC:                        -8.234e+05
Df Model:                          49                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
COP                -2.33e-18   1.08e-20   -214.991      0.000   -2.35e-18   -2.31e-18
company_600028.SS    -0.0015      0.000     -6.846      0.000      -0.002      -0.001
company_601857.SS    -0.0067      0.000    -28.645      0.000      -0.007      -0.006
company_BP           -0.0107      0.000    -52.325      0.000      -0.011      -0.010
company_CNQ           0.0012      0.000      5.597      0.000       0.001       0.002
company_COP          -0.0035      0.000    -17.184      0.000      -0.004      -0.003
company_CVX          -0.0071      0.000    -34.946      0.000      -0.008      -0.007
company_DUK          -0.0093      0.000    -45.564      0.000      -0.010      -0.009
company_E            -0.0113      0.000    -54.994      0.000      -0.012      -0.011
company_ENB          -0.0093      0.000    -43.797      0.000      -0.010      -0.009
company_EOG           0.0032      0.000     15.736      0.000       0.003       0.004
company_EPD          -0.0063      0.000    -29.892      0.000      -0.007      -0.006
company_EQNR         -0.0077      0.000    -35.797      0.000      -0.008      -0.007
company_OXY          -0.0007      0.000     -3.233      0.001      -0.001      -0.000
company_PBR           0.0059      0.000     27.915      0.000       0.006       0.006
company_SHEL         -0.0112      0.000    -54.530      0.000      -0.012      -0.011
company_SLB           0.0016      0.000      7.630      0.000       0.001       0.002
company_SO           -0.0103      0.000    -50.379      0.000      -0.011      -0.010
company_TTE          -0.0112      0.000    -55.040      0.000      -0.012      -0.011
company_XOM          -0.0078      0.000    -37.930      0.000      -0.008      -0.007
DayVar_y              0.9658      0.004    226.365      0.000       0.957       0.974
Year_1995             0.0039      0.000     14.020      0.000       0.003       0.004
Year_1996             0.0046      0.000     17.314      0.000       0.004       0.005
Year_1997             0.0039      0.000     14.173      0.000       0.003       0.004
Year_1998             0.0036      0.000     12.936      0.000       0.003       0.004
Year_1999             0.0028      0.000      9.895      0.000       0.002       0.003
Year_2000             0.0009      0.000      3.002      0.003       0.000       0.001
Year_2001             0.0017      0.000      6.504      0.000       0.001       0.002
Year_2002             0.0019      0.000      7.256      0.000       0.001       0.002
Year_2003             0.0023      0.000      9.281      0.000       0.002       0.003
Year_2004             0.0055      0.000     23.460      0.000       0.005       0.006
Year_2005             0.0102      0.000     44.003      0.000       0.010       0.011
Year_2006             0.0100      0.000     42.950      0.000       0.010

In [34]:
summary_str2 = ls2.summary().as_text()
with open(output + "/OLSsummaryDailyVar.txt", "w") as f:
    f.write(summary_str2)

In [35]:
output

'./output/version15/OPEC_conf_28/fossil_20/before0_after0_normconstsust_34_cleaned'

In [39]:
copdates

,Date,Meeting Title,Start,End
0,2002-03-15,119th Meeting of the OPEC Conference,2002-03-15,2002-03-15
1,2002-06-26,120th (Extraordinary) Meeting of the OPEC Conf...,2002-06-26,2002-06-26
2,2002-09-19,121st Meeting of the OPEC Conference,2002-09-19,2002-09-19
3,2002-12-12,122nd (Extraordinary) Meeting of the OPEC Conf...,2002-12-12,2002-12-12
4,2003-01-12,123rd (Extraordinary) Meeting of the OPEC Conf...,2003-01-12,2003-01-12
...,...,...,...,...
73,2020-06-06,179th Meeting of the OPEC Conference,2020-06-06,2020-06-06
75,2020-11-30,180th Meeting of the OPEC Conference,2020-11-30,2020-11-30
80,2021-07-01,181st Meeting of the OPEC Conference,2021-07-01,2021-07-01
82,2021-12-01,182nd Meeting of the OPEC Conference,2021-12-01,2021-12-01
